In [1]:
import xlrd
import time, requests
from xlwt import Workbook
from random import randint
from time import sleep

In [2]:
fdWB = xlrd.open_workbook("FilteredDatabase.xls") # read workbook
fdSheet = fdWB.sheet_by_index(0)
newWB = xlrd.open_workbook("april16new.xlsx")
newSheet = newWB.sheet_by_index(0)

In [3]:
wb = Workbook() # write workbook 
filteredSheetForWriting = wb.add_sheet("Filtered Sheet")

In [4]:
# Old Database Data retrieval (FDB = Filtered Database)
Title_FDB, Authors_FDB, Journal_FDB, DOI_FDB, Affiliations_FDB = [], [], [], [], []
for i in range(0, fdSheet.nrows):
    if fdSheet.cell_type(i, 0) != xlrd.XL_CELL_EMPTY:
        Title_FDB.append(fdSheet.cell_value(i, 0))
        Authors_FDB.append(fdSheet.cell_value(i, 1))
        Journal_FDB.append(fdSheet.cell_value(i, 2))
        DOI_FDB.append(fdSheet.cell_value(i, 3))
        Affiliations_FDB.append(fdSheet.cell_value(i, 5))

In [5]:
# Write Old Filtered Data
for i in range(len(DOI_FDB)):
    filteredSheetForWriting.write(i, 0, Title_FDB[i])
    filteredSheetForWriting.write(i, 1, Authors_FDB[i])
    filteredSheetForWriting.write(i, 2, Journal_FDB[i])
    filteredSheetForWriting.write(i, 3, DOI_FDB[i])
    filteredSheetForWriting.write(i, 5, Affiliations_FDB[i])

In [6]:
# Filter DOI to be usable
Title, Authors, Journal, DOI = [], [], [], []
for i in range(1, newSheet.nrows):
    if newSheet.cell_type(i, 10) != xlrd.XL_CELL_EMPTY:
        DOIValue = newSheet.cell_value(i, 10) # index 10 is DOI
        # filter DOI to be usable
        DOIOnly = DOIValue.replace("https://doi.org/", "")
        DOIOnly = DOIOnly.replace("http://dx.doi.org/", "")
        DOIOnly = DOIOnly.replace("doi:", "")
        DOIOnly = DOIOnly.split(' ', 1)[0]
        if DOIOnly.startswith("DOI") or DOIOnly.startswith("10"):
            DOI.append(DOIOnly)
            Title.append(newSheet.cell_value(i, 0))
            Authors.append(newSheet.cell_value(i, 1))
            Journal.append(newSheet.cell_value(i, 5))   

In [7]:
# Write Filtered Data
for i in range(len(DOI)):
    filteredSheetForWriting.write(i+len(DOI_FDB), 0, Title[i])
    filteredSheetForWriting.write(i+len(DOI_FDB), 1, Authors[i])
    filteredSheetForWriting.write(i+len(DOI_FDB), 2, Journal[i])
    filteredSheetForWriting.write(i+len(DOI_FDB), 3, DOI[i])

In [8]:
# Begin retrieving data
URLS = []
baseURL = "https://api.crossref.org/v1/works/"
for i in range(len(DOI)):
    URLS.append(baseURL + DOI[i])

In [9]:
# Retrieve JSON Info
pageInfo =[]
for i in range(len(URLS)):
    sleep(randint(0,5))
    page = requests.get(URLS[i])
    if page.status_code == 200:
        pageJson = page.json()
        pageInfo.append(pageJson)
    else:
        pageInfo.append("Response not 200")

In [10]:
for i in range(len(pageInfo)):
    if pageInfo[i] == "Response not 200":
        filteredSheetForWriting.write(i+len(DOI_FDB), 5, "Response not 200")
    elif "author" in pageInfo[i].get("message"):
        if str(pageInfo[i].get("message").get("author")[0].get("affiliation")) == "[]":
            filteredSheetForWriting.write(i+len(DOI_FDB), 5, "No affiliation found")
        else:
            filteredSheetForWriting.write(i+len(DOI_FDB), 5, str(pageInfo[i].get("message").get("author")[0].get("affiliation")[0].get("name")))
    else:
        filteredSheetForWriting.write(i+len(DOI_FDB), 5, "Author Not Found")

In [11]:
wb.save("FilteredDatabase.xls")